[Node 68: Webprogrammierung](http://www-static.etp.physik.uni-muenchen.de/kurs/Computing/python2/node68.html)

Navigation:

**Next:** [Web-Framework Django](node69.ipynb) **Up:** [Web-Framework Django](node69.ipynb) **Previous:** [Web-Framework Django](node69.ipynb)

## Web Programming
For performance reasons, web servers are usually written in C/C++, such as Apache or [lighttpd](https://www.lighttpd.net/). Such a performance is not always necessary, e.g. in embedded systems.

The Python standard library provides two classes derived from <font color=#0000e6> ``Socket.Server.TCPServer``</font> for a web server: <font color=#0000e6> ``SimpleHTTPServer``</font> and <font color=#0000e6> ``CGIHTTPServer``</font> . Both classes are based on the <font color=#0000e6> ``http.server``</font> .

The simplest web server:

In [ ]:
import http.server as hs
server_address = ("", 9090)
handler_class  = hs.BaseHTTPRequestHandler
server_class   = hs.HTTPServer
server = server_class(server_address, handler_class)
server.serve_forever()

Run this program and go to the following address in a web browser: [http://localhost:9090/](http://localhost:9090/).

This doesn't work properly yet: The server can't interpret the <font color=#0000e6> ``GET``</font> method because the <font color=#0000e6> ``handle``</font> method of <font color=#0000e6> ``BaseHTTPRequestHandler``</font> could not find a <font color=#0000e6> ``do_GET``</font> method.

A corresponding extension is necessary:

In [ ]:
import http.server as hs
class MyHandler(hs.BaseHTTPRequestHandler):
    def do_GET(self):
        print("Got GET Request from", self.client_address)
        self.wfile.write(b'Sorry, I do not speak HTTP. Go away.\r\n')
server_address = ("", 9092)
handler_class  = MyHandler
server_class   = hs.HTTPServer
server = server_class(server_address, handler_class)
server.serve_forever()

Now calling http://localhost:9091/ works.

(Note: As before, we use a different port each time to bypass [this problem](https://dev.to/dechamp/the-dreaded-bind-address-already-in-use-kill-it-583l). There are plenty of them. Another, more elegant solution is to catch the exception and release the socket again -- see below.)

This example can now be extended to a simple pocket calculator, for example. This accepts requests of the form "<tt>/{add,sub,mul,div}/num1/num2</tt>" that are passed in the URL. This request allows to add, subtract etc. two numbers <tt>num1</tt> and <tt>num2</tt>.

The example also performs a series of checks to ensure that the request is valid. This is essential for services that are open to the world so as not to be vulnerable to abuse. (Typically, a server that you start on your own computer will not be easily accessible worldwide, but only from your home network.)

Additionally, ``server.serv_forever()`` is wrapped in a ``try-except`` block to cleanly stop the server (and unblock the port again).

In [ ]:
import http.server

class CalcHandler(http.server.BaseHTTPRequestHandler):

  def do_GET(self):
    path = self.path

    lst = path.split("/")
    if len(lst) != 4:
      self.send_response(403)
      self.end_headers()
      self.wfile.write(b"Illegal syntax. Use /{add,sub,mul,div}/num1/num2\r\n")
      return

    dummy, op, arg1, arg2 = lst

    if op not in ("add", "sub", "mul", "div"):
      self.send_response(403)
      self.end_headers()
      self.wfile.write(b"Illegal operation: %s\r\n" % op)
      return

    try:
      numarg1 = float(arg1)
      numarg2 = float(arg2)
    except ValueError:
      self.send_response(403)
      self.end_headers()
      self.wfile.write(b"Numerical arguments expected\r\n")
      return

    if op == "add":
      result = numarg1 + numarg2
    elif op == "sub":
      result = numarg1 - numarg2
    elif op == "mul":
      result = numarg1 * numarg2
    elif op == "div":
      if numarg2 == 0:
        result = "NaN"
      else:
        result = numarg1 / numarg2

    self.send_response(200)
    self.end_headers()
    self.wfile.write(str(result).encode())


def run_server(port=9093):
  server_class = http.server.HTTPServer
  server_address = ("", port)
  handler_class = CalcHandler

  server = server_class(server_address, handler_class)
  try:
    print("Starting server...")
    server.serve_forever()
  except KeyboardInterrupt:
    print("Stopping server...")
    server.socket.close()
  except: # something else went wrong
    raise

if __name__ == "__main__":
  run_server()

This can be reached at http://localhost:9092.

---

The <font color=#0000ff> **SimpleHTTPServer**</font> offers a little more web server functionality, with which you can display directories and transfer files:

In [ ]:
import http.server
            
def run_server(port=9093):
    server_class   = http.server.HTTPServer
    handler_class  = http.server.SimpleHTTPRequestHandler
    server_address = ("", port)
    
    server = server_class(server_address, handler_class)
    try:
        server.serve_forever()
    except KeyboardInterrupt:
        server.socket.close()
    except:
        raise
    
if __name__ == '__main__':
    run_server()

Access address: http://localhost:9093

The next step is not only to display content, but also to run scripts.
The easiest way to do this is with so-called <font color=#0000ff> **CGI scripts**</font> using `CGIHTTPServer`. CGI scripts placed in the <font color=#0000e6> ``cgi-bin``</font> subdirectory of the web server can be used to run programs on the web server and transmit the results to the client. However, this is tricky from a security point of view. The scripts can be written in any programming language, not just Python (as long as it's server-side supported).

A <font color=#0000ff> **CGIHTTPServer**</font>:

In [ ]:
import http.server
            
def run_server(port=9094):
    server_class   = http.server.HTTPServer
    handler_class  = http.server.CGIHTTPRequestHandler
    handler_class.cgi_directories = ['/source', '/source/cgi-bin']
    server_address = ("", port)
    
    server = server_class(server_address, handler_class)
    try:
        server.serve_forever()
    except KeyboardInterrupt:
        server.socket.close()
    except:
        raise
        
if __name__ == '__main__':
    run_server()

We place the following CGI script with the name <tt>cgiprintenv.py</tt> in the subdirectory <tt>source/cgi-bin/</tt>:
```python
#!/usr/bin/env python3

import os
from sys import stdout

stdout.write("Content-type: text/plain\r\n\r\n")

for key in sorted(os.environ.keys()):
    print("%s=%s" % (key, os.environ[key]))
```
and make it executable (`chmod +x cgiprintenv.py`).

Then you can call it up in a web browser at the following address: http://localhost:9094/source/cgi-bin/cgiprintenv.py

A further elegant CGI solution is the [**WSGI-Standard**](https://de.wikipedia.org/wiki/Web_Server_Gateway_Interface), which will not be discussed further here.

